<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=dea9aace0b2da80341d093ba149246fc7069160f6f71c8e589f6032e7bc988d0
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-27 10:17:09
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.42 C
-------------------
Today PnL: 47.45 K (0.33%)
Current PnL: -19.25 L (-12.82%)
CY Booked + Current PnL: -7.15 L (-4.76%)
-------------------
Total profit:  2.43 L
Total loss:  -21.68 L
-------------------
Total Booked + Current PnL: 19.83 L (16.11%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.5%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 86.37 L (60.65%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.53%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.0,-7.66,72.0,X-LC,3.50,231098.0,20732.0,8042.0,2.05,9.86,3.48,13.68,37.0,2.58,1.63,28.01,XY25,NTT,REFINERIES
85,VOLTAS,1530.0,2.28,57.0,X-MC,3.09,214725.0,22983.0,14773.0,-0.69,11.99,6.88,19.69,99.0,1.56,1.51,19.60,XY25,NTT,AC
78,TTKPRESTIG,770.0,97.01,51.0,M-SC,2.06,84862.0,-15915.0,16005.0,0.45,-15.79,18.86,0.09,245.0,-0.99,0.60,11.49,OX40N,NTT,DURABLES
42,ITC,452.0,-36.66,66.0,X-LC,1.72,204915.0,4777.0,16106.0,0.54,2.39,7.86,10.44,4.0,0.30,1.44,8.08,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-9.64,41.0,X-MC,3.21,89325.0,-1353.0,16257.0,-0.52,-1.49,18.20,16.44,101.0,-0.08,0.63,18.38,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SATIN,274.00,-22.47,70.0,H-SC,4.17,237917.0,-38367.0,181578.0,5.43,-13.89,76.32,51.83,126.0,-0.21,1.68,15.90,XY24,NTT,FINANCE
12,BANKINDIA,190.00,-18.01,72.0,H-MC,15.03,210414.0,30606.0,81809.0,2.17,17.02,38.88,62.52,88.0,0.37,1.48,56.91,XR,NTT,BANKS
58,RELIANCE,1533.00,-7.66,72.0,X-LC,3.50,231098.0,20732.0,8042.0,2.05,9.86,3.48,13.68,37.0,2.58,1.63,28.01,XY25,NTT,REFINERIES
15,BSOFT,836.99,-11.70,67.0,H-SC,14.34,99208.0,-41445.0,114228.0,1.99,-29.47,115.14,51.74,131.0,-0.36,0.70,14.78,XR,ATH,IT
17,CAMS,5211.76,-2.73,60.0,X-SC,0.62,110617.0,8613.0,35309.0,1.81,8.44,31.92,43.06,122.0,0.24,0.78,27.00,X40N,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.0,-45.67,49.0,L-SC,27.41,81312.0,-32237.0,72286.0,-1.94,-28.39,88.90,35.27,268.0,-0.45,0.57,99.01,XR,NTT,HOTELS
82,VAIBHAVGBL,521.0,53.17,57.0,H-SC,4.52,132805.0,-49970.0,161039.0,-1.37,-27.34,121.26,60.77,125.0,-0.31,0.94,18.95,XR,NTT,JEWELLERY
83,VALIANTORG,838.0,-286.53,40.0,H-SC,8.37,103700.0,-59905.0,181216.0,-1.29,-36.62,174.75,74.15,139.0,-0.33,0.73,32.93,XR,NTT,CHEMICALS
70,SURYODAY,240.0,68.86,61.0,H-SC,13.00,154795.0,-24276.0,89998.0,-1.28,-13.56,58.14,36.71,135.0,-0.27,1.09,53.59,XR,NTT,BANKS
38,INDIGOPNTS,1408.0,93.08,34.0,M-SC,4.87,126604.0,-47955.0,47983.0,-1.12,-27.47,37.90,0.02,221.0,-1.00,0.89,10.12,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.0,41.94,48.0,M-MC,9.72,224153.0,-809.0,168473.0,0.55,-0.36,75.16,74.53,192.0,-0.00,1.58,31.78,XY24,BTT,STEEL
79,UJJIVANSFB,60.0,124.04,69.0,H-SC,16.13,141210.0,-1269.0,20786.0,0.54,-0.89,14.72,13.70,163.0,-0.06,1.00,66.67,OX40N,NTT,BANKS
36,IEX,219.0,-30.16,62.0,H-SC,17.20,199655.0,-299.0,97531.0,0.05,-0.15,48.85,48.63,115.0,-0.00,1.41,11.39,XR,NTT,MISC
56,RECLTD,446.0,45.36,50.0,M-LC,5.70,203885.0,715.0,41409.0,0.73,0.35,20.31,20.74,55.0,0.02,1.44,7.24,XY25,NTT,FINANCE
87,WIPRO,420.0,-13.79,49.0,M-LC,5.89,151863.0,918.0,108536.0,0.81,0.61,71.47,72.51,53.0,0.01,1.07,6.67,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,124.04,69.0,H-SC,16.13,141210.0,-1269.0,20786.0,0.54,-0.89,14.72,13.70,163.0,-0.06,1.00,66.67,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.87,58.0,H-SC,2.55,228096.0,-41571.0,77895.0,-0.21,-15.42,34.15,13.47,138.0,-0.53,1.61,15.98,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,97.01,51.0,M-SC,2.06,84862.0,-15915.0,16005.0,0.45,-15.79,18.86,0.09,245.0,-0.99,0.60,11.49,OX40N,NTT,DURABLES
73,TATAELXSI,9161.0,-21.53,58.0,H-MC,7.31,106210.0,-25625.0,67847.0,0.83,-19.44,63.88,32.03,98.0,-0.38,0.75,18.96,OX40N,NTT,IT
67,SIS,528.0,2063.21,51.0,H-SC,3.18,87299.0,-23733.0,47342.0,-0.31,-21.37,54.23,21.26,156.0,-0.50,0.62,17.95,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,50.0,M-LC,5.70,203885.0,715.0,41409.0,0.73,0.35,20.31,20.74,55.0,0.02,1.44,7.24,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.80,53.0,M-LC,10.97,211714.0,6638.0,105921.0,-0.00,3.24,50.03,54.89,52.0,0.06,1.49,28.11,X5K,ATH,METALS
46,KPIGREEN,731.05,8.25,67.0,H-SC,5.22,129037.0,3740.0,55189.0,-1.09,2.98,42.77,47.03,141.0,0.07,0.91,61.57,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,61.0,M-LC,9.29,163330.0,22490.0,22001.0,1.33,15.97,13.47,31.59,66.0,1.02,1.15,46.69,XY24,NTT,BANKS
87,WIPRO,420.00,-13.79,49.0,M-LC,5.89,151863.0,918.0,108536.0,0.81,0.61,71.47,72.51,53.0,0.01,1.07,6.67,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-18.01,72.0,H-MC,15.03,210414.0,30606.0,81809.0,2.17,17.02,38.88,62.52,88.0,0.37,1.48,56.91,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-39.40,62.0,M-SC,3.61,102435.0,10936.0,67812.0,-0.35,11.95,66.20,86.07,223.0,0.16,0.72,48.75,XR,NTT,DURABLES
46,KPIGREEN,731.05,8.25,67.0,H-SC,5.22,129037.0,3740.0,55189.0,-1.09,2.98,42.77,47.03,141.0,0.07,0.91,61.57,MH,ATH,POWER
32,HINDZINC,730.22,29.80,53.0,M-LC,10.97,211714.0,6638.0,105921.0,-0.00,3.24,50.03,54.89,52.0,0.06,1.49,28.11,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.27,51.0,H-SC,6.98,127693.0,4357.0,127272.0,1.62,3.53,99.67,106.72,119.0,0.03,0.90,27.74,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.83,16.0,X-LC,15.22,146398.0,-130125.0,240195.0,-0.63,-47.06,164.07,39.81,54.0,-0.54,1.03,3.19,XY24,NTT,AUTO
24,DMART,5391.45,-16.90,36.0,X-LC,10.10,101611.0,-975.0,27780.0,0.47,-0.95,27.34,26.13,38.0,-0.04,0.72,24.38,X40N,ATH,FMCG
53,QUESS,986.00,-47.66,39.0,X-SC,38.69,51903.0,-13103.0,164029.0,1.24,-20.16,316.03,232.18,198.0,-0.08,0.37,1.24,XY24,NTT,MISC
2,ABBOTINDIA,35195.00,-9.64,41.0,X-MC,3.21,89325.0,-1353.0,16257.0,-0.52,-1.49,18.20,16.44,101.0,-0.08,0.63,18.38,X40,ATH,PHARMA
51,PAGEIND,51769.93,-27.05,42.0,X-MC,6.89,82400.0,-1110.0,21144.0,0.43,-1.33,25.66,23.98,82.0,-0.05,0.58,4.41,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-1.97,50.0,H-LC,0.90,157500.0,-28595.0,76072.0,-0.07,-15.37,48.30,25.51,15.0,-0.38,1.11,16.07,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,66.0,X-LC,1.72,204915.0,4777.0,16106.0,0.54,2.39,7.86,10.44,4.0,0.30,1.44,8.08,X40,NTT,FMCG
52,PGHH,17907.65,-29.36,43.0,X-MC,3.08,204315.0,3495.0,64298.0,0.51,1.74,31.47,33.76,80.0,0.05,1.44,7.03,X40,ATH,FMCG
29,HAVELLS,2069.16,-6.46,44.0,X-MC,3.08,231847.0,-15884.0,90930.0,-0.58,-6.41,39.22,30.30,92.0,-0.17,1.63,6.03,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,2.28,57.0,X-MC,3.09,214725.0,22983.0,14773.0,-0.69,11.99,6.88,19.69,99.0,1.56,1.51,19.60,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,39.0,X-SC,38.69,51903.0,-13103.0,164029.0,1.24,-20.16,316.03,232.18,198.0,-0.08,0.37,1.24,XY24,NTT,MISC
74,TATAMOTORS,1065.00,-53.83,16.0,X-LC,15.22,146398.0,-130125.0,240195.0,-0.63,-47.06,164.07,39.81,54.0,-0.54,1.03,3.19,XY24,NTT,AUTO
21,COLPAL,3767.14,-4.76,44.0,X-MC,8.12,222560.0,-40805.0,154145.0,-0.62,-15.49,69.26,43.04,84.0,-0.26,1.57,3.32,XY25,ATH,FMCG
3,ACC,3906.00,-38.07,48.0,X-MC,2.84,184550.0,-53401.0,206050.0,-0.25,-22.44,111.65,64.15,174.0,-0.26,1.30,3.52,XY24,BTT,CEMENT
10,BAJAJHFL,181.50,-14.48,46.0,X-MC,5.80,180318.0,-20188.0,117351.0,-0.24,-10.07,65.08,48.45,90.0,-0.17,1.27,3.98,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,39.0,X-SC,38.69,51903.0,-13103.0,164029.0,1.24,-20.16,316.03,232.18,198.0,-0.08,0.37,1.24,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.05,42.0,X-MC,6.89,82400.0,-1110.0,21144.0,0.43,-1.33,25.66,23.98,82.0,-0.05,0.58,4.41,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.01,46.0,X-SC,4.65,83134.0,-62026.0,137953.0,-0.45,-42.73,165.94,52.31,136.0,-0.45,0.59,11.28,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.64,41.0,X-MC,3.21,89325.0,-1353.0,16257.0,-0.52,-1.49,18.20,16.44,101.0,-0.08,0.63,18.38,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.29,47.0,X-SC,4.36,96728.0,-31942.0,77237.0,0.03,-24.82,79.85,35.20,219.0,-0.41,0.68,11.02,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-27.08,59.0,X-LC,12.04,290300.0,-55656.0,122361.0,0.82,-16.09,42.15,19.28,1.0,-0.45,2.05,7.32,X40,ATH,IT
40,INFY,2275.00,-17.09,61.0,X-LC,6.88,322588.0,9700.0,161971.0,0.81,3.10,50.21,54.87,3.0,0.06,2.27,11.64,X40,BTT,IT
42,ITC,452.00,-36.66,66.0,X-LC,1.72,204915.0,4777.0,16106.0,0.54,2.39,7.86,10.44,4.0,0.30,1.44,8.08,X40,NTT,FMCG
84,VBL,671.64,-18.84,50.0,X-LC,9.81,292204.0,-23638.0,136313.0,-0.74,-7.48,46.65,35.67,5.0,-0.17,2.06,5.33,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,49.0,H-LC,6.16,249264.0,-12355.0,131562.0,0.23,-4.72,52.78,45.57,7.0,-0.09,1.76,6.06,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7011.11,60.0,L-SC,7.29,78036.0,-15774.0,92395.0,-0.54,-16.81,118.40,81.67,269.0,-0.17,0.55,51.12,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,27.41,81312.0,-32237.0,72286.0,-1.94,-28.39,88.90,35.27,268.0,-0.45,0.57,99.01,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,50.0,H-SC,9.20,91860.0,-6120.0,27723.0,-0.65,-6.25,30.18,22.05,152.0,-0.22,0.65,33.53,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-39.40,62.0,M-SC,3.61,102435.0,10936.0,67812.0,-0.35,11.95,66.20,86.07,223.0,0.16,0.72,48.75,XR,NTT,DURABLES
46,KPIGREEN,731.05,8.25,67.0,H-SC,5.22,129037.0,3740.0,55189.0,-1.09,2.98,42.77,47.03,141.0,0.07,0.91,61.57,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-197.73,79.0,M-SC,26.55,168471.0,19251.0,124921.0,-0.56,12.90,74.15,96.62,208.0,0.15,1.19,82.32,XY25,NTT,FINANCE
5,ANGELONE,3033.00,17.86,73.0,X-SC,5.63,208846.0,17840.0,39869.0,1.29,9.34,19.09,30.21,157.0,0.45,1.47,32.17,X40N,NTT,FINANCE
68,SONACOMS,806.63,-32.59,70.0,M-SC,8.68,83948.0,-17211.0,57211.0,0.21,-17.01,68.15,39.54,202.0,-0.30,0.59,18.37,AR,ATH,AUTO
59,REPCOHOME,880.00,-55.29,72.0,H-SC,3.48,258108.0,-28397.0,285726.0,-0.08,-9.91,110.70,89.82,134.0,-0.10,1.82,36.56,XY24,NTT,FINANCE
46,KPIGREEN,731.05,8.25,67.0,H-SC,5.22,129037.0,3740.0,55189.0,-1.09,2.98,42.77,47.03,141.0,0.07,0.91,61.57,MH,ATH,POWER


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.39
1,25,43.83
2,50,74.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.53
LC    32.72
MC    22.75
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.31
X40      16.00
XY25     12.28
XR       11.30
X40N      9.32
AR        7.91
OX40N     7.60
X200      1.79
X5K       1.49
SR        1.09
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.73
X-LC    21.87
X-MC    16.98
M-SC    12.67
M-LC     5.15
X-SC     4.68
H-LC     4.66
H-MC     3.88
M-MC     1.58
L-SC     1.45
L-LC     1.04
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.63,-3.90,37.81
IT,12.77,-16.03,78.26
FINANCE,12.31,-8.26,57.34
MISC,6.94,-16.72,81.40
BANKS,6.66,-5.73,61.35
PAINTS,5.61,-16.02,33.31
ELECTRICAL,5.29,-9.78,48.29
INSURANCE,3.89,-0.31,37.71
AUTO,2.80,-44.74,105.15


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3272981.0,22
XR,1307089.0,14
AR,1166090.0,9
X40,774227.0,12
X40N,608485.0,8
OX40N,568215.0,10
XY25,442698.0,8
SR,266219.0,2
X5K,105921.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3345602.0,25
M-SC,1207391.0,15
X-MC,1125231.0,13
X-LC,1036766.0,13
X-SC,518038.0,6
H-MC,329011.0,3
M-LC,277867.0,4
H-LC,277796.0,3
L-SC,254432.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1150249.0      6
           AR         834413.0      5
           XR         777218.0      7
M-SC       XY24       694569.0      6
X-MC       XY24       585875.0      4
X-LC       X40        443903.0      5
           XY24       308430.0      2
X-SC       X40N       276772.0      4
H-SC       SR         266219.0      2
           OX40N      262314.0      4
X-MC       X40        253087.0      6
X-LC       X40N       214362.0      3
H-LC       AR         207634.0      2
H-MC       XY24       179355.0      1
X-MC       XY25       168918.0      2
M-MC       XY24       168473.0      1
L-SC       XR         164681.0      2
X-SC       XY24       164029.0      1
M-SC       OX40N      148303.0      4
           XY25       124921.0      1
           AR         124043.0      2
X-MC       X40N       117351.0      1
M-SC       XR         115555.0      2
M-LC       XR         108536.0      1
           X5K        105921.0      1
L-SC       OX40N       89751.0      1
H-MC       XR          81809.0      1
X-SC       X40         77237.0      1
H-LC       X200        70162.0      1
X-LC       XY25        70071.0      3
H-MC       OX40N       67847.0      1
L-MC       XR          59290.0      1
H-SC       MH          55189.0      1
M-LC       XY25        41409.0      1
L-LC       XY25        37379.0      1
M-LC       XY24        22001.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
